# CPU Load vs Performance

In [22]:
import pandas as pd

# GET

In [2]:
getFolder = '../get'
interfolders = ['ACME', 'mobius', 'om2m-mongo']
testsFolders = ['Locust1', 'Locust2', 'Locust3', 'Locust4']

In [18]:
mainFolder = getFolder + '/' + interfolders[0] + '/' + testsFolders[0]

# Read the csv file and  _stats_history.csv
stats_df = pd.read_csv(mainFolder + '/_stats_history.csv')
docker_df = pd.read_csv(mainFolder + '/docker_stats_output.csv')

# Convert timetamp in stats_df to start from 0
stats_df['Timestamp'] = stats_df['Timestamp'] - stats_df['Timestamp'][0]

# Find the index where user count first becomes non-zero
first_non_zero_index = stats_df[stats_df['User Count'] != 0].index.min()

# Find the index where user count becomes zero again after it became non-zero
zero_index = stats_df[stats_df.index > first_non_zero_index][stats_df['User Count'] == 0].index.min()

# remove all rows after the user count became zero again
stats_df = stats_df[stats_df.index < zero_index]

# Create a DataFrame with the desired time range
min_time = docker_df['time'].min()
max_time = docker_df['time'].max()
desired_time_range = range(min_time, max_time + 1)
desired_df = pd.DataFrame({'time': desired_time_range})

# Merge the two DataFrames
final_docker_df = pd.merge(desired_df, docker_df, on='time', how='left', suffixes=('', 'ffill'))

# Fill the NaN values with the previous values
final_docker_df = final_docker_df.fillna(method='ffill')

/tmp/ipykernel_44628/1443310276.py:14: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/tmp/ipykernel_44628/1443310276.py:29: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [33]:
# merge the two DataFrames on the Timestamp = time
final_df = pd.merge(stats_df, final_docker_df, left_on='Timestamp', right_on='time', how='left', suffixes=('', 'ffill'))
# Plot User Count, CPU Usage, Memory Usage against time
# Remove % from CPU and Memory Usage
# use plotly express to plot
times = final_df['Timestamp']
users = final_df['User Count']
cpu = [i.replace('%', '') for i in final_df['CPU %']]
memory = [i.replace('%', '') for i in final_df['MEM %']]
# Requests/s
requests = final_df['Requests/s']
# 50% 
# cpu50 = final_df['50%'].tolist()
# 99%
cpu99 = final_df['99%'].tolist()

# Plot time vs user count and time vs cpu usage and time vs memory usage
import plotly.graph_objects as go

# fig = go.Figure()
# fig.add_trace(go.Scatter(x=times, y=users, mode='lines', name='User Count'))
# fig.add_trace(go.Scatter(x=times, y=cpu, mode='lines', name='CPU Usage'))
# fig.add_trace(go.Scatter(x=times, y=memory, mode='lines', name='Memory Usage'))
# fig.add_trace(go.Scatter(x=times, y=requests, mode='lines', name='Requests/s'))
# # fig.add_trace(go.Scatter(x=times, y=cpu50, mode='lines', name='50%'))
# fig.add_trace(go.Scatter(x=times, y=cpu99, mode='lines', name='99%'))

# fig.update_layout(title='User Count, CPU Usage, Memory Usage, Requests/s, 50%, 99% vs Time',
#                     xaxis_title='Time (s)',
#                     yaxis_title='User Count, CPU Usage, Memory Usage, Requests/s, 50%, 99%')
# fig.show()

# 1. User Count vs CPU Usage, Memory Usage
# 2. Requests/s vs CPU Usage, Memory Usage
# 3. 99% vs CPU Usage, Memory Usage
# Make multiple plots
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=users, y=cpu, mode='lines', name='CPU Usage'))
fig1.add_trace(go.Scatter(x=users, y=memory, mode='lines', name='Memory Usage'))
fig1.update_layout(title='User Count vs CPU Usage, Memory Usage',
                    xaxis_title='User Count',
                    yaxis_title='CPU Usage, Memory Usage')

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=requests, y=cpu, mode='lines', name='CPU Usage'))
fig2.add_trace(go.Scatter(x=requests, y=memory, mode='lines', name='Memory Usage'))
fig2.update_layout(title='Requests/s vs CPU Usage, Memory Usage',
                    xaxis_title='Requests/s',
                    yaxis_title='CPU Usage, Memory Usage')

fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=cpu99, y=cpu, mode='lines', name='CPU Usage'))
fig3.add_trace(go.Scatter(x=cpu99, y=memory, mode='lines', name='Memory Usage'))
fig3.update_layout(title='99% vs CPU Usage, Memory Usage',
                    xaxis_title='99%',
                    yaxis_title='CPU Usage, Memory Usage')

fig1.show()
fig2.show()
fig3.show()

